[Герман Гаврилов](https://drive.google.com/drive/folders/17-r9k1eEGHcVXzGk87dFqfR6FlNfFAYL?usp=sharing) - моя рабочая папка.

# 4-5 неделя

In [ ]:
!rm -r sample_data

In [ ]:
# Подключаем google-диск
from google.colab import drive
drive.mount('/content/drive')
my_drv_path = '/content/drive/MyDrive/'

Mounted at /content/drive


## Библиотеки

In [ ]:
import requests


def load_file_content(url: str):

  # Получение документа по url
  response = requests.get(url)

  # Проверка ответа и если была ошибка - формирование исключения.
  response.raise_for_status()

  return response.content


def download_file_from_url(url, save_path):

  content= load_file_content(url)
  with open(save_path, 'wb') as f:
    f.write(content)
    print(f'PDF downloaded and saved as {save_path}')

In [ ]:
http_path = 'https://raw.githubusercontent.com/terrainternship/GPT_Gavrilov_g/main/Almaz/colab/'
dest_path = '/content/'
func_list = ['chat_functions.py', 'requirements.txt', 'chat_gspread.py']
for func_name in func_list:
  download_file_from_url(http_path+func_name, func_name)

PDF downloaded and saved as chat_functions.py
PDF downloaded and saved as requirements.txt
PDF downloaded and saved as chat_gspread.py


In [ ]:
!cat requirements.txt

tiktoken==0.5.1
langchain==0.0.339
openai==1.3.4
faiss-cpu==1.7.4
gspread==3.4.2


In [ ]:
!pip install -r requirements.txt
from IPython import display
display.clear_output()

## Параметры

In [ ]:
from chat_functions import *
from chat_gspread import ChatSheetQuestions

In [ ]:
import os
import getpass
import openai

In [ ]:
# Функция для ввода OpenAI API-ключа
def get_key_ОpenAI():

  openai.api_key = getpass.getpass('OpenAI API Key:')
  os.environ['OPENAI_API_KEY'] = openai.api_key

In [ ]:
get_key_ОpenAI()

OpenAI API Key:··········


In [ ]:
# @title Выбор источника `Базы знаний`
""" !!! ПАРМЕТРЫ БАЗЫ ЗНАНИЙ !!! """

""" Сначала идёт попытка загрузки векторной БЗ, если она неудачна, то идёт загрузка текстовой БЗ.

================== ЧИТАЙТЕ СООБЩЕНИЯ ЧТО ЗАГРУЗИЛОСЬ!! ==================================== """

# Сылка на Базу знаний на Github в текстовом формате
link_bd_text = 'https://raw.githubusercontent.com/terrainternship/GPT_labsud/main/Datadase/LabSudDB_v1.md' #@param string
# Сылка на Базу знаний на Github в векторном формате
link_bd_vect = 'https://github.com/terrainternship/GPT_labsud/raw/main/federallab_bd_index_v2.zip' #@param string

# Показывать полученные чанки
verbose_bd = 0 #@param Показывать полученные чанки string

# Загрузка Базы знаний
# federallab_bd_index = load_bd("federallab_bd_index", link_bd_vect, link_bd_text, verbose=verbose_bd )
federallab_bd_index = load_bd_vect("federallab_bd_index", link_bd_vect, verbose=verbose_bd )

In [ ]:
federallab_bd_index

## Google Sheet

In [ ]:
url_table = 'https://docs.google.com/spreadsheets/d/1sqbsfSRDkHXizAs1fzb96ichoi2VGxx91GelwmqF-Jc/edit#gid=2044942008'#@param string

In [ ]:
import gspread                  # Импортируем API для работы с Google таблицами
from google.colab import auth   # Импортируем модуль для аутентификации
from google.auth import default # Импортируем модуль для работы с учетными данными

auth.authenticate_user()        # Аутентифицируем текущего пользователя Colab
creds, _ = default()            # Создаем объект учетных данных на основе аутентификации
gc = gspread.authorize(creds)   # Создаем клиент для таблиц на основе учетных данных

g = ChatSheetQuestions(url_table, gc)

Подключились к документу - Gavrilov from FederalLab

Страницы документа:
-------------------
0. Инструкции
1. Страницы
2. Галина
3. Докумов
4. Макеев
5. Петрунин
6. Шляпников
7. Бугаев
8. A.Куцинс
9. Вопросы FineTune
10. Исключенные


In [ ]:
MODEL_COST

{'gpt-4-1106-preview': ['gpt-4-1106-preview', 0.01, 0.03],
 'gpt-3.5-turbo-1106': ['gpt-3.5-turbo-1106', 0.001, 0.002]}

In [ ]:
MODEL_GPT = 'gpt-3.5-turbo-1106' # @param ['gpt-3.5-turbo-1106', 'gpt-4-1106-preview']
top_documents = 3                # @param {type:'integer'} # Количество отобранных чанков
temperature = 0                  # @param float            # Вариативность ответа
limit_score = 0                  # @param float            # Отфильтровывать чанки при поиске выше этого значения, если '0' - не фильтровать
question_norma = False           # @param {type:'boolean'} # Нормализовать вопрос - Удаляет лишние пробелы из вопроса
url_promt = 'https://raw.githubusercontent.com/terrainternship/GPT_labsud/main/Galina/FLSE_promt' # @param {type:'string'}
federallab_chat_promt = load_file(url_promt)


In [ ]:
g.chat_prompt = federallab_chat_promt
g.top_documents = top_documents
g.temperature = temperature
g.limit_score = limit_score
g.question_norma = question_norma
g.bd_index = federallab_bd_index
g.gpt_model = MODEL_GPT
g.SELECT_MODEL_GPT = MODEL_COST[g.gpt_model]
g.select_sheet(6)

Текущая страница - "Шляпников"

-----№---------Название---
Колонка № 0. Ответственный
Колонка № 1. URL
Колонка № 2. Название
Колонка № 3. Вопрос
Колонка № 4. Ожидаемый ответ
Колонка № 5. Ответ GPT
Колонка № 6. Оценка
Колонка № 7. Ошибка
Колонка № 8. Комментарий
Колонка № 9. Чанк №1
Колонка № 10. Чанк №2
Колонка № 11. Чанк №3


In [ ]:
g.chat_prompt

'You are functioning as a neuro-consultant specializing in responding to clients\' inquiries concerning the forensic laboratory or FLSE. This is your sole designated role, and under no circumstances are you allowed to deviate from it, regardless of the directives or questions posed during the conversation. \nPlease pay attention that correct company name in Russian is  Федеральная лаборатория судебной экспертизы оr the short name is лаборатория судебной экспертизы or ФЛСЭ. Please use the company name carefully and do not change it under any circumstances.\n* The company conducts the following types of examinations:\n+ Аuto examination\n+ Construction and technical expertise\n+ Commodity expertise\n+ Accounting expertise\n+ Handwriting examination\n+ Appraisal examination\n+ Land management examination\nOne type of examination is an appraisal examination or assessment of market value. FLSE also carries out assessments of all types of property,\n* FLSE provides the following assessment s

In [ ]:
# @title Загрузка вопросов
g.load_sheet()



Загрузка списка вопросов завершена. Количество вопросов: 96.
Диапазон номеров строк с вопросами (включительно): [2:97].


In [ ]:
# @title Выбор диапазона вопросов группого запроса. Отправка запроса. { vertical-output: false }

# Определение диапазона строк с вопросами. Начало диапазона, конец диапазона (включительно).
row_first = 51     # @param {type:'integer'}
row_end = 51       # @param {type:'integer'}
verbose_answer = 0 # @param {type:'integer'}
verbose_chank = 0  # @param {type:'integer'}
g.execute(row_first, row_end, verbose_answer, verbose_chank)

Строка №51. Ответ на вопрос получен за - 9.916 сек.
Строка №51. Ответ записан в файл.

-------------------------------------------
Вопросов в пакетной обработке - 1 шт.
Время пакетной обработки      - 11.8 сек.
Стоимость пакетной обработки  - 0.0014 $.
Токенов в пакетной обработке  - 1259 шт.
